In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, roc_curve, roc_auc_score

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import GlobalMaxPooling1D

In [3]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils  import to_categorical

In [4]:
df=pd.read_csv("Appliance_5.csv")

In [5]:
df.head()

,Occupancy,Appliance ID,Sin(time),Cos(time),Sin(day),Cos(day),Power consumption,Normalized power,P(t)-P(t-1),P(t)-P(t+1),Class Label
0,1,5,0.00000,1.00000,0.0,1.0,1406,0.55322,1406,335,1
1,0,5,0.25882,0.96593,0.0,1.0,1071,0.36711,-335,361,4
2,0,5,0.50000,0.86603,0.0,1.0,710,0.16655,-361,-80,4
3,0,5,0.70711,0.70711,0.0,1.0,790,0.21100,80,69,4
4,0,5,0.86603,0.50000,0.0,1.0,721,0.17266,-69,-638,4


In [6]:
df.columns

Index(['Occupancy', 'Appliance ID', 'Sin(time)', 'Cos(time)', 'Sin(day)',
       'Cos(day)', 'Power consumption', 'Normalized power', 'P(t)-P(t-1)',
       'P(t)-P(t+1)', 'Class Label'],
      dtype='object')

In [7]:
shuffle(df)

,Occupancy,Appliance ID,Sin(time),Cos(time),Sin(day),Cos(day),Power consumption,Normalized power,P(t)-P(t-1),P(t)-P(t+1),Class Label
733,0,5,-2.588200e-01,-9.659300e-01,0.49378,0.869590,558,0.082107,-273,-722,4
10750,1,5,-5.000000e-01,8.660300e-01,0.98735,0.158560,1030,0.344330,-93,521,0
12109,1,5,-2.588200e-01,-9.659300e-01,0.68077,-0.732490,1366,0.531000,675,430,3
15444,1,5,1.220000e-16,-1.000000e+00,-0.99732,0.073095,1486,0.597660,-9,854,3
3366,0,5,1.000000e+00,6.120000e-17,0.66806,-0.744100,1296,0.492110,735,744,4
...,...,...,...,...,...,...,...,...,...,...,...
1296,0,5,0.000000e+00,1.000000e+00,0.80136,0.598180,808,0.221000,808,-638,1
212,1,5,-8.660300e-01,5.000000e-01,0.13728,0.990530,964,0.307660,401,-523,0
5321,0,5,-9.659300e-01,-2.588200e-01,-0.61528,-0.788310,717,0.170440,-366,-441,4
2112,1,5,0.000000e+00,1.000000e+00,0.99844,0.055917,0,-0.227890,-1496,-541,2


In [8]:
# Split the data into train and test sets
train_size = int(len(df)*0.8)
train, test = df.iloc[:train_size],df.iloc[train_size:]

In [9]:
# Split train and test sets into predictors (X) and outcome (y) variables
x_train = train.drop('Class Label', axis = 1)
y_train = train.loc[:,['Class Label']]
x_test = test.drop('Class Label', axis = 1)
y_test = test.loc[:,['Class Label']]

In [10]:
# Scale the X data 
scaler_x = MinMaxScaler(feature_range = (0,1))
input_scaler = scaler_x.fit(x_train)
train_x_norm = input_scaler.transform(x_train)
test_x_norm = input_scaler.transform(x_test)

In [11]:
# One-hot encode y data
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [12]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((14016, 10), (3504, 10), (14016, 5), (3504, 5))

In [13]:
x_train=np.array(x_train)
y_train=np.array(y_train)
x_test=np.array(x_test)
y_test=np.array(y_test)

In [14]:
x_train = x_train.reshape(x_train.shape[0],x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

In [15]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((14016, 10, 1), (3504, 10, 1), (14016, 5), (3504, 5))

In [16]:
verbose, epochs, batch_size = 1, 50, 32
n_timesteps, n_features = x_train.shape[1], x_train.shape[2]

In [17]:
# Define the model architecture
model = Sequential()
model.add(Conv1D(filters=16, kernel_size=3, activation='relu', input_shape=(10, 1)))
model.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
model.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(5, activation='softmax'))

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 8, 16)             64        
                                                                 
 conv1d_1 (Conv1D)           (None, 6, 16)             784       
                                                                 
 conv1d_2 (Conv1D)           (None, 4, 16)             784       
                                                                 
 global_max_pooling1d (Globa  (None, 16)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 64)                1088      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                        

In [19]:
#compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
#fit the model
model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)

Epoch 1/50
438/438 [==============================] - 1s 1ms/step - loss: 2.5373 - accuracy: 0.4490
Epoch 2/50
438/438 [==============================] - 0s 1ms/step - loss: 0.8540 - accuracy: 0.4935
Epoch 3/50
438/438 [==============================] - 0s 1ms/step - loss: 0.7996 - accuracy: 0.5236
Epoch 4/50
438/438 [==============================] - 0s 1ms/step - loss: 0.7837 - accuracy: 0.5298
Epoch 5/50
438/438 [==============================] - 0s 1ms/step - loss: 0.7435 - accuracy: 0.5487
Epoch 6/50
438/438 [==============================] - 0s 1ms/step - loss: 0.6861 - accuracy: 0.5599
Epoch 7/50
438/438 [==============================] - 0s 1ms/step - loss: 0.6559 - accuracy: 0.5731
Epoch 8/50
438/438 [==============================] - 0s 1ms/step - loss: 0.6322 - accuracy: 0.5791
Epoch 9/50
438/438 [==============================] - 0s 1ms/step - loss: 0.6131 - accuracy: 0.5925
Epoch 10/50
438/438 [==============================] - 0s 1ms/step - loss: 0.6085 - accuracy: 0.5930

In [21]:
_, accuracy = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=verbose)
#  return accuracy

110/110 [==============================] - 0s 765us/step - loss: 0.1182 - accuracy: 0.9572


In [22]:
accuracy

0.9571917653083801

In [23]:
prediction = model.predict(x_test)

110/110 [==============================] - 0s 658us/step


In [38]:
prediction

array([[1.77031146e-15, 1.00000000e+00, 1.90574546e-25, 1.01329145e-09,
        3.23881979e-15],
       [9.92503047e-01, 6.56168004e-06, 1.89118066e-06, 7.47865671e-03,
        9.77081618e-06],
       [1.46171305e-05, 1.18876997e-09, 1.21560245e-11, 9.79179276e-07,
        9.99984384e-01],
       ...,
       [8.36974848e-03, 1.67778376e-06, 1.99653538e-08, 9.91625488e-01,
        3.13239116e-06],
       [4.39021193e-07, 1.09164393e-10, 6.22274872e-16, 9.99999523e-01,
        1.01221937e-08],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        0.00000000e+00]], dtype=float32)

In [25]:
y_test

array([[0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [26]:
# prediction_NN = np.argmax(prediction[1], axis=-1)

In [27]:
prediction_NN=np.argmax(prediction, axis=-1)

In [28]:
prediction_NN

array([1, 0, 4, ..., 3, 3, 2], dtype=int64)

In [29]:
y_test1 = test.loc[:,['Class Label']]

In [30]:
y_test1=np.array(y_test1)

In [31]:
y_test1

array([[1],
       [0],
       [4],
       ...,
       [3],
       [3],
       [2]], dtype=int64)

In [32]:
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(y_test1, prediction_NN))

Accuracy =  0.9571917808219178


In [33]:
#Confusion Matrix - verify accuracy of each class
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test1, prediction_NN)
print(cm)

[[1336    0    0   81    0]
 [   0   88    0    0    0]
 [   0    0   88    0    0]
 [   0    0    0  499    0]
 [   0    0    0   69 1343]]


In [34]:
from sklearn.metrics import classification_report

# print(class_labels)
target_names = ['Class 0', 'Class 1', 'Class 2', 'Class 3', 'Class 4']
print(classification_report(y_test1, prediction_NN, target_names=target_names,digits=3))

              precision    recall  f1-score   support

     Class 0      1.000     0.943     0.971      1417
     Class 1      1.000     1.000     1.000        88
     Class 2      1.000     1.000     1.000        88
     Class 3      0.769     1.000     0.869       499
     Class 4      1.000     0.951     0.975      1412

    accuracy                          0.957      3504
   macro avg      0.954     0.979     0.963      3504
weighted avg      0.967     0.957     0.959      3504



In [35]:
# !pip install mlxtend

In [36]:
# from mlxtend.plotting import plot_confusion_matrix

In [37]:
# class_dict = {0: 'Class 0',
#               1: 'Class 1',
#               2: 'Class 2',
#               3: 'Class 3',
#               4: 'Class 4'}

# multiclass = np.array([[9926,   67,   19,   56,    0],
#                        [178,  855,    0,   41,   40],
#                        [268,    0,  848,    0,    0],
#                        [27,   16,    0, 2326,   15],
#                        [0,    1,    0,   36, 6305]])

# fig, ax = plot_confusion_matrix(
#     conf_mat=multiclass,
#     class_names=class_dict.values(),show_absolute=False,show_normed=True,colorbar=True)